## Using MCMC to fit real data

### AY 128/256 (UC Berkeley, 2021-2024)

* Let's look at a famous dataset
<img src="hubble_diagram.jpg" width="90%">
* Hubble (1929) finds that more distant galaxies have larger positive velocities than closer galaxies
* Ends up being a big deal
* What were some of our crisiticms of this plot, and by extension the result, from our first meeing?

## Our challenge
* Hubble found $H_0 \sim 500$ km/s/Mpc, more or less fitting data with no error bars by eye.
* Let's see how well this result stands up to more detailed analysis.

## Step 1: Data Wrangling
* We need to get Hubble's data
* Fortunately, it's in Table 1 of Hubble (1929)
<img src="hubble_table.jpeg" width="80%">

 - $m_s$ = photographic magnitude of brightest stars involved.

 - $r$ = distance in units of $10^6$ parsecs. The first two are Shapley’s values.

 - $v$ = measured velocities in km/s.    NGC 6822, 221, 224 and 5457 are recent determinations by Humason.

 - $m_t$ = Holetschek’s visual magnitude as corrected by Hopmann. The first three objects were not measured by Holetschek, and the values of mt represent estimates by the author based upon such data as are available.

 - $M_t$ = total visual absolute magnitude computed from m_t and $r$.

* Since it's a short table, I turned this into a csv file by hand.  It contains galaxy name, distance, velocity, and absolute magnitude.

## Step 1: Data Visualization
* Let's use Pandas to read in the data and try to re-create Hubble's plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 4]

# read in the csv file; column names are in the file already
df = pd.read_csv('hubble1929.tab1')

import pylab as plt
# plot distance and velocity
plt.scatter(df['distance'], df['velocity'])

In [ ]:
# Let's make this plot look a bit nicer using Seaborn
import seaborn as sns
sns.set(style="whitegrid", color_codes=True, font_scale=1.3)

cmap = plt.get_cmap('plasma')

plt.scatter(df['distance'], df['velocity'], c=df['absMag'], s=150, edgecolor='0.7', cmap=cmap)
plt.xlabel('Distance (Mpc)', size=24)
plt.ylabel('Velocity (km/s)', size=24)

cb = plt.colorbar()
cb.set_label("Absolute Magnitude", rotation=270, labelpad=20, fontsize=18)

# make colorbar brighest to faintest
cb.ax.invert_yaxis()


## Step 3a: Write down a model for the data
* $y_n = m~x_n + b$
 - Linear model to descibe velocity as a function of distance
 - 2 parameters to fit for: $m$ (slope) and $b$ (offset)
 - $y_n$: velocity of the nth galaxy; $x_n$: distance to the nth galaxy
 - We have no error bars (i.e., no $\sigma_n$) -- is this a problem? Why or why not?

## Step 3b: Error bars
* Let's fit for the scatter in the data (or the size of the typical uncertainty)
* Assume errors are normally distributed
* error term: $lnsig = \log(\sigma)$
* why use $\log(\sigma)$ instead of $\sigma$ as a parameter?

## Aside: Review Bayes's theorem
* Bayes's theorem: $P(\theta \mid \{y_n\}) \propto P(\{y_n\} \mid \theta) \, P(\theta)$
 - $\theta = \{m,b,\log(\sigma)\}$
* Assume we have $N$ independent data points (i.e., galaxies in this case).  The posterior for the entire dataset is the product of the posteriors for each data point.

 - $P(\theta \mid \{y_n\}) = \prod_n^N P(\theta \mid y_n) \propto\ \prod_n^N P( y_n \mid \theta)\, P(\theta)$ 
 

* Or in log-space:
 - $\log P(\theta \mid \{y_n\}) \propto \sum_n^N \log P(y_n \mid \theta) + \log P(\theta)$
 - why work in log-space?

## Step 4: Write down priors and log-likelihood

* Uniform prior on $m$:

 $$
P(m)=
\begin{cases}
1 \, \rm{if} \, 100 < m < 1000\\
0 \, {\rm otherwise}\\
\end{cases}
$$

* Uniform prior on $b$:

 $$
P(b)=
\begin{cases}
1 \, \rm{if} -500 < b < 500\\
0 \, {\rm otherwise}\\
\end{cases}
$$

* Uniform prior on $\log(\sigma)$:

 $$
P(b)=
\begin{cases}
1 \, \rm{if} -10 < \log(\sigma) < 10\\
0 \, {\rm otherwise}\\
\end{cases}
$$
 

* Likelihood function for a normal distribution:

\begin{equation}
P(\{y_n\} \mid \{m, b, \log(\sigma)\}) = \frac{1}{\sqrt{2\pi\sigma}} e^{-\frac{(y_n - mx_n - b)^2}{2 \sigma_n^2}}
\end{equation}

   - where $x_n$ is shorthand for the 'distance' variable


* log-Likelihood function for a normal distribution:

 $ \log P(\{y_n\} \mid \{m, b, \log(\sigma)\} = -\frac{1}{2} \sum^{N}_{n=1} \Big(\frac{(y_n - mx_n - b)^2}{\sigma_n^2} + \log(2 \pi \sigma_n^2) \Big)$
 
   - how might this change if we had values for $\sigma_n$ already?

Another way of writing this model:
* $m \sim {\rm Uniform}(100, 1000)$
* $b \sim {\rm Uniform}(-500, 500)$
* lnsig = $\log(\sigma) \sim {\rm Uniform}(-10,10)$
* $y_n \sim {\rm Normal}(m x_n +b, \sigma)$

## Steps 5 & 6: Implement and run the model in python using PyMC, make some diagnostic plots

We'll use the built-in Normal to create a likelihood. To define and use your own likelihood (eg. log-likelihood) you can do a Custom function in pymc. See https://www.pymc.io/projects/examples/en/latest/howto/blackbox_external_likelihood_numpy.html#introduction
    

In [ ]:
import pymc as pm
import arviz as az
from arviz import plot_trace as traceplot

# set up the model
with pm.Model() as model:
    
    # define priors
    m = pm.Uniform("m", lower=100, upper=1000)
    b = pm.Uniform("b", lower=-500, upper=500)
    lnsig = pm.Uniform("lnsig", lower=-10, upper=10)
    
    likelihood = pm.Normal("obs", mu = m*df['distance'].values+b, 
              sigma=np.exp(lnsig), observed=df['velocity'].values)

    # now set up the model to run
    # default of PyMC is to use the no-turn sampler (NUTS)
    
    # pm.sample will run the sampler and store output in 'trace' 
    trace = pm.sample(draws=5000, tune=1000, chains=4, cores=4, discard_tuned_samples=True)
    
    # traceplot is a routine for plotting the 'traces' from the samples
    _ = traceplot(trace, var_names=["m", "b", "lnsig"])


In [ ]:
# pm.summary provides some useful summary and convergance statistics
az.summary(trace, var_names=["m", "b", "lnsig"])

`corner` is a nice routine for visualizing the marginalized posterior distributions

  - 1d plots (1d histograms) are called **marginalized distributions**
  - 2d plots (2d histograms) are called **joint distributions**


In [ ]:
import corner

# make the corner plot and plot results from Hubble's paper as 'truth'
# overplot percentiles: 16, 50, 84 on 1d historgrams
_ = corner.corner(trace, truths=[500, 0, None], quantiles=[.16, .50, .84], show_titles=True,
    title_kwargs={"fontsize": 12})

# how big is the scatter/typical error in the data?
# let's use the median of logsig for this
print(f"The typical error in velocity is ~{int(np.median(np.exp(trace.posterior['lnsig'])))} km/s")

## Step 7: Posterior Predictive check
* the technical term that means to plot your model on top of the data and do a sanity check

In [ ]:
# Let's make this plot look a bit nicer using Seaborn

#import seaborn as sns
sns.set(style="whitegrid", color_codes=True, font_scale=1.3)
plt.scatter(df['distance'], df['velocity'], c=df['absMag'], s=150, edgecolor='0.7', cmap=cmap)
plt.xlabel('Distance (Mpc)', size=24)
plt.ylabel('Velocity (km/s)', size=24)
cb = plt.colorbar()
cb.set_label("Absolute Magnitude", rotation=270, labelpad=20, fontsize=18)
cb.ax.invert_yaxis()
np.random.seed(42)


# this code loops over the trace array and pulls 500 random sets of m,b, and logsig
# it then generates 500 posterior models (i.e., lines in velocity and distance space) and overplots them on the data


m_model = trace.posterior['m'].values.flatten()
b_model = trace.posterior['b'].values.flatten()
dist = np.asarray(df['distance'])


for i in np.random.randint(0, len(m_model)-1, 500):
    plt.plot(dist, m_model[i]*dist + b_model[i], color="k", lw=0.1, alpha=0.3, zorder=0)